In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.B.500.250204/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/B/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/RNA_count_matrix/B.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.B.2500.250324/"
celltype = "B"

In [3]:
n.cores = 8

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

Prepare SCENT meta data

In [9]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
s1d1_TAGTTGTCACCCTCAC-1,s1d1_TAGTTGTCACCCTCAC-1,1701,31,B,7.438972,0.018224574
s1d1_TGGATTGGTTTGCGAA-1,s1d1_TGGATTGGTTTGCGAA-1,16740,295,B,9.725556,0.017622461
s1d1_GTAAGGTCAATAACCT-1,s1d1_GTAAGGTCAATAACCT-1,2041,52,B,7.621195,0.025477707
s1d1_GGATTATGTTACATCC-1,s1d1_GGATTATGTTACATCC-1,5014,78,B,8.519989,0.015556442
s1d1_AACGACAAGTTGTCAA-1,s1d1_AACGACAAGTTGTCAA-1,1477,21,B,7.297768,0.014218009
s1d1_AGCAATACAATATGGA-1,s1d1_AGCAATACAATATGGA-1,2295,42,B,7.738488,0.018300654
s1d1_GGGAGTAGTGATTTGG-1,s1d1_GGGAGTAGTGATTTGG-1,2708,106,B,7.903966,0.039143279
s1d1_CCATAATCATGTCAGC-1,s1d1_CCATAATCATGTCAGC-1,2064,26,B,7.632401,0.012596899
s1d1_ATCAGGTTCATGACCG-1,s1d1_ATCAGGTTCATGACCG-1,1802,27,B,7.496652,0.014983352


RUN SCENT prediction

In [10]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
                bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [11]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

Warning message in dir.create(paste(dir.output, "chr", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.B.2500.250324//chr' already exists”


In [12]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

In [13]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.05]
  pairs.E2G.chr.keep = pairs.E2G.chr[!pairs.E2G.chr$PairName %in% pairs.E2G.chr.res$PairName]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr7"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr8"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [14]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [15]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 158027 ranges and 11 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
  chr1.chr1     chr1 100949328-100950217      * |      CDC14A
  chr1.chr1     chr1 101127580-101129049      * |      TRMT13
  chr1.chr1     chr1 101235454-101237644      * |      CDC14A
  chr1.chr1     chr1 101309029-101310358      * |      CDC14A
  chr1.chr1     chr1   10428101-10428351      * |       KIF1B
        ...      ...                 ...    ... .         ...
  chrX.chrX     chrX       989777-990609      * |       ASMTL
  chrX.chrX     chrX       989777-990609      * |       IL3RA
  chrX.chrX     chrX       989777-990609      * |       P2RY8
  chrX.chrX     chrX     9981069-9982168      * |       TBL1X
  chrX.chrX     chrX     9981069-9982168      * |        WWC3
                          PeakName               PairName       beta        se
                       <character>            <character

In [16]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100949328,100950217,CDC14A,B,0.15838060,0.06509202,2.433180,1.496686e-02,0.0408,1.389340,1,1.389340,0.30799186
chr1,101127580,101129049,TRMT13,B,-0.34531849,0.12994156,-2.657491,7.872475e-03,0.0424,1.372634,-1,-1.372634,0.31597948
chr1,101235454,101237644,CDC14A,B,0.12745242,0.04433975,2.874451,4.047308e-03,0.0232,1.634512,1,1.634512,0.22024669
chr1,101309029,101310358,CDC14A,B,-0.13088904,0.04847740,-2.700001,6.933919e-03,0.0408,1.389340,-1,-1.389340,0.30799186
chr1,10428101,10428351,KIF1B,B,0.27783519,0.11093193,2.504556,1.226053e-02,0.0208,1.681937,1,1.681937,0.20648041
chr1,10474745,10475415,SLC25A33,B,0.22740185,0.09001422,2.526288,1.152749e-02,0.0408,1.389340,1,1.389340,0.30799186
chr1,1058559,1060138,GNB1,B,0.09744853,0.03700676,2.633263,8.456880e-03,0.0248,1.605548,1,1.605548,0.22909158
chr1,1063702,1064446,SLC35E2B,B,-0.37271462,0.11850638,-3.145102,1.660291e-03,0.0048,2.318759,-1,-2.318759,0.08249370
chr1,1068464,1070398,SSU72,B,-0.17439776,0.08350311,-2.088518,3.675111e-02,0.0504,1.297569,-1,-1.297569,0.35251186


In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges